In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime


# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind"
label_col = "label"

# 포함할 abnormal return 컬럼
abn_return_cols = [
    "abn_ret_minus_10m", "abn_ret_minus_20m", "abn_ret_minus_30m", 
    "abn_ret_minus_40m", "abn_ret_minus_50m", "abn_ret_minus_60m",
    "abn_ret_10m", "abn_ret_20m", "abn_ret_30m", 
    "abn_ret_40m", "abn_ret_50m", "abn_ret_60m"
]

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_before_telegram = pd.read_sql(query, engine)
df_before_telegram["period_dummy"] = 0
df_before_telegram

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_10m,abn_ret_minus_20m,abn_ret_minus_30m,abn_ret_minus_40m,abn_ret_minus_50m,abn_ret_minus_60m,abn_ret_10m,abn_ret_20m,abn_ret_30m,abn_ret_40m,abn_ret_50m,abn_ret_60m,label,period_dummy
0,2021-01-04 10:08:00+00:00,-2.14,-2.14,-2.14,NaN,NaN,NaN,-0.45,0.17,0.05,0.15,-0.15,0.18,0,0
1,2021-01-04 10:08:00+00:00,-2.14,-2.14,-2.14,NaN,NaN,NaN,-0.45,0.17,0.05,0.15,-0.15,0.18,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-1.91,-0.66,-1.47,-1.37,-0.60,-0.95,1,0
3,2021-01-04 10:14:00+00:00,-8.40,NaN,NaN,NaN,NaN,NaN,-4.90,-4.20,-3.46,-5.53,-5.16,-6.00,1,0
4,2021-01-04 10:16:00+00:00,0.22,1.55,1.55,1.55,NaN,NaN,0.03,-0.83,-0.54,-0.96,-0.57,-1.83,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,2022-06-29 14:04:00+00:00,2.30,2.19,2.24,2.43,2.88,2.47,0.55,0.43,-0.82,-0.84,-0.85,-1.63,1,0
5068,2022-06-29 14:20:00+00:00,-0.16,0.05,0.02,0.06,0.08,0.17,-0.03,0.34,0.11,0.32,0.54,0.24,1,0
5069,2022-06-29 14:21:00+00:00,0.00,0.12,0.06,-0.14,-0.51,-0.28,0.06,-0.04,-0.11,-0.32,0.03,NaN,1,0
5070,2022-06-29 14:35:00+00:00,0.14,0.27,0.27,0.19,0.18,0.06,-0.22,0.03,0.03,-0.08,NaN,NaN,1,0


In [ ]:
label_table = "label"
return_table = "abnormal_return"
label_col = "label"


all_sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"k.{label_col}"]
)
all_sql_columns_str = ",\n    ".join(all_sql_columns)

query = f"""
SELECT {all_sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
JOIN disclosure_events de ON ar.event_id = de.id
WHERE k.{label_col} IS NOT NULL
ORDER BY ar.event_ts ASC
"""

df_after_telegram = pd.read_sql(query, engine)
df_after_telegram["period_dummy"] = 1
label_map = {0: 1, 1: 0, 2: -1}
df_after_telegram['label'] = df_after_telegram['label'].map(label_map)
df_after_telegram

,event_ts,abn_ret_minus_10m,abn_ret_minus_20m,abn_ret_minus_30m,abn_ret_minus_40m,abn_ret_minus_50m,abn_ret_minus_60m,abn_ret_10m,abn_ret_20m,abn_ret_30m,abn_ret_40m,abn_ret_50m,abn_ret_60m,label,period_dummy
0,2022-07-01 01:03:17+00:00,-1.80,-1.61,-2.03,-2.74,-2.32,-1.18,-1.27,-2.41,-2.36,-1.79,-2.98,-1.98,1,1
1,2022-07-01 01:24:45+00:00,-1.57,-1.62,-1.91,-1.33,-2.52,-1.48,0.53,2.43,1.19,1.57,1.38,1.72,1,1
2,2022-07-01 01:27:02+00:00,-0.05,0.39,0.10,-1.01,-0.14,-0.14,0.29,0.24,1.25,1.39,2.02,1.63,1,1
3,2022-07-01 01:43:15+00:00,-0.89,-0.92,-0.82,-0.81,-0.59,0.16,0.08,0.36,0.31,0.12,-0.18,0.23,1,1
4,2022-07-01 02:30:09+00:00,-1.91,-0.73,0.47,0.44,0.08,1.81,-0.40,-1.02,-1.80,-1.24,-1.82,-1.60,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8076,2023-12-28 05:53:08+00:00,-0.08,0.80,0.02,0.26,0.25,0.86,0.24,0.24,NaN,NaN,NaN,NaN,1,1
8077,2023-12-28 05:55:03+00:00,-0.31,0.84,0.80,1.16,0.76,0.93,-2.10,-3.25,NaN,NaN,NaN,NaN,0,1
8078,2023-12-28 05:56:27+00:00,0.05,0.08,-0.02,-0.16,-0.06,0.12,-0.28,-0.54,NaN,NaN,NaN,NaN,1,1
8079,2023-12-28 05:56:32+00:00,0.05,0.08,-0.02,-0.16,-0.06,0.12,-0.28,-0.54,NaN,NaN,NaN,NaN,1,1


In [4]:
df_total = pd.concat([df_before_telegram, df_after_telegram])



In [16]:
df_total

,event_ts,abn_ret_minus_10m,abn_ret_minus_20m,abn_ret_minus_30m,abn_ret_minus_40m,abn_ret_minus_50m,abn_ret_minus_60m,abn_ret_10m,abn_ret_20m,abn_ret_30m,abn_ret_40m,abn_ret_50m,abn_ret_60m,label,period_dummy
0,2021-01-04 10:08:00+00:00,-2.14,-2.14,-2.14,NaN,NaN,NaN,-0.45,0.17,0.05,0.15,-0.15,0.18,0,0
1,2021-01-04 10:08:00+00:00,-2.14,-2.14,-2.14,NaN,NaN,NaN,-0.45,0.17,0.05,0.15,-0.15,0.18,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-1.91,-0.66,-1.47,-1.37,-0.60,-0.95,1,0
3,2021-01-04 10:14:00+00:00,-8.40,NaN,NaN,NaN,NaN,NaN,-4.90,-4.20,-3.46,-5.53,-5.16,-6.00,1,0
4,2021-01-04 10:16:00+00:00,0.22,1.55,1.55,1.55,NaN,NaN,0.03,-0.83,-0.54,-0.96,-0.57,-1.83,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8076,2023-12-28 05:53:08+00:00,-0.08,0.80,0.02,0.26,0.25,0.86,0.24,0.24,NaN,NaN,NaN,NaN,1,1
8077,2023-12-28 05:55:03+00:00,-0.31,0.84,0.80,1.16,0.76,0.93,-2.10,-3.25,NaN,NaN,NaN,NaN,0,1
8078,2023-12-28 05:56:27+00:00,0.05,0.08,-0.02,-0.16,-0.06,0.12,-0.28,-0.54,NaN,NaN,NaN,NaN,1,1
8079,2023-12-28 05:56:32+00:00,0.05,0.08,-0.02,-0.16,-0.06,0.12,-0.28,-0.54,NaN,NaN,NaN,NaN,1,1


# 회귀분석

### 📘 모형식

$$
\left| CAR_{post,i} - CAR_{pre,i} \right| = \alpha + \beta \cdot Telegram_i + \epsilon_i
$$

---

### 📊 변수 설명

**종속변수 (Dependent Variable)**  
이벤트 전후 CAR 변화폭의 절댓값:

$$
\left| CAR_{post,i} - CAR_{pre,i} \right|
$$


**설명변수 (Key Independent Variable)**  
텔레그램 도입 여부 변수:

$$
Telegram_i =
\begin{cases}
1, & \text{텔레그램 도입 이후 (After introduction)} \\
0, & \text{도입 이전 (Before introduction)}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{텔레그램 도입 이후와 이전 간 CAR 변화폭의 차이가 없다.}) \\
H_1 &: \beta < 0 \quad &(\text{텔레그램 도입 이후 CAR 변화폭이 감소했다.})
\end{aligned}
$$
---

### 📈 해석

이 회귀모형은 **텔레그램 도입 여부**가 이벤트 전후 누적초과수익률(CAR) 변화폭의 크기에 미치는 영향을 분석하기 위한 것입니다.  
β < 0 이고 통계적으로 유의하다면 공시 정보가 도입 이후 시장에 보다 신속하게 반영되어 정보 비효율성이 감소했음을 시사하며, 텔레그램 공시 알림 서비스가 시장 효율성 향상에 실질적으로 기여했음을 보여줍니다.


In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# df_total이 이미 존재한다고 가정
horizons = [10, 20, 30, 40, 50, 60]
rows = []

for h in horizons:
    post_col = f"abn_ret_{h}m"
    pre_col  = f"abn_ret_minus_{h}m"
    if post_col not in df_total.columns or pre_col not in df_total.columns:
        raise KeyError(f"'{post_col}' 또는 '{pre_col}' 컬럼을 찾을 수 없습니다.")

    y = (df_total[post_col] - df_total[pre_col]).abs()
    X = sm.add_constant(df_total["period_dummy"], has_constant="add")

    valid = ~(y.isna() | X.isna().any(axis=1))
    y_ = y[valid]
    X_ = X[valid]

    model = sm.OLS(y_, X_).fit(cov_type="HC1")

    beta = model.params.get("period_dummy", np.nan)
    se   = model.bse.get("period_dummy", np.nan)
    tval = model.tvalues.get("period_dummy", np.nan)
    pval = model.pvalues.get("period_dummy", np.nan)

    rows.append({
        "horizon_min": h,
        "coef_period_dummy": beta,
        "std_err": se,
        "t_value": tval,
        "p_value": pval,
        "sig_0.05": "*" if (not pd.isna(pval) and pval < 0.05) else "",
        "n_obs": int(model.nobs),
        "r_squared": model.rsquared
    })

result_table = pd.DataFrame(rows).sort_values("horizon_min").reset_index(drop=True)

# ✅ p-value 포맷 조정 (지수표기 없이 소수점 8자리, 0은 "<1e-8"로 표시)
result_table["p_value"] = result_table["p_value"].apply(
    lambda x: f"{x:.12f}" if (x > 0 and not pd.isna(x)) else "<1e-8"
)

# 보기 좋게 출력
display_cols = ["horizon_min", "coef_period_dummy", "std_err", "t_value", "p_value", "sig_0.05", "n_obs", "r_squared"]
print(result_table[display_cols].to_string(index=False))

 horizon_min  coef_period_dummy  std_err   t_value        p_value sig_0.05  n_obs  r_squared
          10          -0.284615 0.033934 -8.387221 0.000000000000        *  12329   0.005971
          20          -0.278708 0.038763 -7.190126 0.000000000001        *  11454   0.004773
          30          -0.268489 0.043548 -6.165326 0.000000000703        *  10596   0.003830
          40          -0.444945 0.057169 -7.782917 0.000000000000        *   9728   0.006982
          50          -0.396699 0.058895 -6.735650 0.000000000016        *   8895   0.005539
          60          -0.371746 0.063477 -5.856352 0.000000004731        *   8069   0.004634


# 방향 정합성

### 📘 모형식 정의


$$
Pr(hit_{i,h} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해

$$
h \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}
$$



$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- β > 0 이고 **통계적으로 유의**하다면  → “정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다.


In [22]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

df = df_total.copy()
df["label_sign"] = df["label"]  # 라벨 정의에 따라 그대로 사용 (1=긍정, -1=부정, 0=중립)

def logistic_hit(h):
    delta = df[f"abn_ret_{h}m"] - df[f"abn_ret_minus_{h}m"]
    realized_sign = np.sign(delta)
    mask = df["label_sign"] != 0  # 중립 제외
    hit = (realized_sign[mask] == df.loc[mask, "label_sign"]).astype(int)
    X = sm.add_constant(df.loc[mask, "period_dummy"])
    m = sm.Logit(hit, X).fit(disp=0)
    return m.params["period_dummy"], m.pvalues["period_dummy"]

results = []
for h in [10, 20, 30, 40, 50, 60]:
    beta, p = logistic_hit(h)
    sig = "*" if p < 0.05 else ""
    results.append({"h": h, "beta": beta, "p-value": p, "sig_0.05": sig})

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

 h      beta      p-value sig_0.05
10 -0.357103 5.514973e-15        *
20 -0.316421 1.064956e-12        *
30 -0.338652 1.256642e-14        *
40 -0.372399 2.047281e-17        *
50 -0.316588 7.089432e-13        *
60 -0.327579 2.225846e-13        *


In [24]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

df = df_total.copy()
# 실제 라벨 정의: 1(긍정), -1(부정), 0(중립)
df["label_sign"] = df["label"]

# ✅ 중립 제거
df_nn = df[df["label_sign"] != 0].copy()

# 샘플 구성 요약 출력
n_total = len(df)
n_neutral = (df["label_sign"] == 0).sum()
n_used = len(df_nn)
pos_cnt = (df_nn["label_sign"] == 1).sum()
neg_cnt = (df_nn["label_sign"] == -1).sum()

print("=== Sample summary (neutral removed) ===")
print(f"Total events:        {n_total:,}")
print(f"Removed neutrals:    {n_neutral:,}")
print(f"Used (non-neutral):  {n_used:,}")
print(f"  - Positive (1):    {pos_cnt:,}")
print(f"  - Negative (-1):   {neg_cnt:,}")
print()

def logistic_hit(df_subset, h):
    """
    df_subset: 중립 제거된 데이터프레임 (label_sign ∈ {+1, -1})
    h: 시간 창 (분)
    반환: beta, p, odds_ratio, p0(before), p1(after), diff_pp, n
    """
    delta = df_subset[f"abn_ret_{h}m"] - df_subset[f"abn_ret_minus_{h}m"]
    realized_sign = np.sign(delta)

    # hit: 감성 부호와 실제 부호가 일치하면 1
    hit = (realized_sign == df_subset["label_sign"]).astype(int)

    # 설명변수: 상수 + period_dummy (도입 이후=1)
    X = sm.add_constant(df_subset["period_dummy"])

    # 로지스틱 회귀 적합
    m = sm.Logit(hit, X).fit(disp=0)

    beta = float(m.params["period_dummy"])
    pval = float(m.pvalues["period_dummy"])
    odds_ratio = float(np.exp(beta))

    # 도입 전/후 예측 정합률(평균)
    X0 = X.copy(); X0["period_dummy"] = 0
    X1 = X.copy(); X1["period_dummy"] = 1
    p0 = float(m.predict(X0).mean())
    p1 = float(m.predict(X1).mean())
    diff_pp = (p1 - p0) * 100.0

    return {
        "h": h,
        "beta": beta,
        "p_value": pval,
        "odds_ratio": odds_ratio,
        "p_before": p0,
        "p_after": p1,
        "diff_pp": diff_pp,
        "n_obs": int(m.nobs)
    }

# 창별 결과
results = []
for h in [10, 20, 30, 40, 50, 60]:
    results.append(logistic_hit(df_nn, h))

results_df = pd.DataFrame(results)

# 보기 좋게 반올림 및 유의성 표시
results_df["sig_0.05"] = np.where(results_df["p_value"] < 0.05, "*", "")
print(results_df[["h","beta","p_value","sig_0.05","odds_ratio","p_before","p_after","diff_pp","n_obs"]]
      .round({"beta":4, "p_value":8, "odds_ratio":3, "p_before":3, "p_after":3, "diff_pp":2}))


=== Sample summary (neutral removed) ===
Total events:        13,153
Removed neutrals:    3,690
Used (non-neutral):  9,463
  - Positive (1):    6,995
  - Negative (-1):   2,468

    h    beta  p_value sig_0.05  odds_ratio  p_before  p_after  diff_pp  n_obs
0  10 -0.3571      0.0        *       0.700     0.667    0.584    -8.34   9463
1  20 -0.3164      0.0        *       0.729     0.608    0.531    -7.74   9463
2  30 -0.3387      0.0        *       0.713     0.561    0.476    -8.43   9463
3  40 -0.3724      0.0        *       0.689     0.527    0.434    -9.27   9463
4  50 -0.3166      0.0        *       0.729     0.468    0.391    -7.74   9463
5  60 -0.3276      0.0        *       0.721     0.430    0.352    -7.78   9463


In [27]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

df = df_total.copy()
# 실제 라벨 정의: 1(긍정), -1(부정), 0(중립)
df["label_sign"] = df["label"]

# ✅ 중립 제거
df_nn = df[df["label_sign"] != 0].copy()

# 샘플 구성 요약 출력
n_total = len(df)
n_neutral = (df["label_sign"] == 0).sum()
n_used = len(df_nn)
pos_cnt = (df_nn["label_sign"] == 1).sum()
neg_cnt = (df_nn["label_sign"] == -1).sum()

print("=== Sample summary (neutral removed) ===")
print(f"Total events:        {n_total:,}")
print(f"Removed neutrals:    {n_neutral:,}")
print(f"Used (non-neutral):  {n_used:,}")
print(f"  - Positive (1):    {pos_cnt:,}")
print(f"  - Negative (-1):   {neg_cnt:,}")
print()

def logistic_hit(df_subset, h):
    """
    df_subset: 중립 제거된 데이터프레임 (label_sign ∈ {+1, -1})
    h: 시간 창 (분)
    반환: beta, p, odds_ratio, p0(before), p1(after), diff_pp, n
    """
    # ✅ post CAR 기준: 공시 이후 (0 → +h) 누적초과수익의 부호만 사용
    post = df_subset[f"abn_ret_{h}m"]
    realized_sign = np.sign(post)

    # hit: 감성 부호와 post CAR 부호가 일치하면 1
    hit = (realized_sign == df_subset["label_sign"]).astype(int)

    # 설명변수: 상수 + period_dummy (도입 이후=1)
    X = sm.add_constant(df_subset["period_dummy"])

    # 로지스틱 회귀 적합
    m = sm.Logit(hit, X).fit(disp=0)

    beta = float(m.params["period_dummy"])
    pval = float(m.pvalues["period_dummy"])
    odds_ratio = float(np.exp(beta))

    # 도입 전/후 예측 정합률(평균)
    X0 = X.copy(); X0["period_dummy"] = 0
    X1 = X.copy(); X1["period_dummy"] = 1
    p0 = float(m.predict(X0).mean())
    p1 = float(m.predict(X1).mean())
    diff_pp = (p1 - p0) * 100.0

    return {
        "h": h,
        "beta": beta,
        "p_value": pval,
        "odds_ratio": odds_ratio,
        "p_before": p0,
        "p_after": p1,
        "diff_pp": diff_pp,
        "n_obs": int(m.nobs)
    }

# 창별 결과
results = []
for h in [10, 20, 30, 40, 50, 60]:
    results.append(logistic_hit(df_nn, h))

results_df = pd.DataFrame(results)

# 보기 좋게 반올림 및 유의성 표시
results_df["sig_0.05"] = np.where(results_df["p_value"] < 0.05, "*", "")
print(results_df[[
    "h","beta","p_value","sig_0.05","odds_ratio","p_before","p_after","diff_pp","n_obs"
]].round({
    "beta":4, "p_value":8, "odds_ratio":3, "p_before":3, "p_after":3, "diff_pp":2
}))


=== Sample summary (neutral removed) ===
Total events:        13,153
Removed neutrals:    3,690
Used (non-neutral):  9,463
  - Positive (1):    6,995
  - Negative (-1):   2,468

    h    beta   p_value sig_0.05  odds_ratio  p_before  p_after  diff_pp  \
0  10  0.1994  0.000006        *       1.221     0.429    0.478     4.94   
1  20  0.2077  0.000003        *       1.231     0.398    0.448     5.06   
2  30  0.1462  0.001071        *       1.157     0.385    0.420     3.51   
3  40  0.0813  0.069693                1.085     0.381    0.401     1.94   
4  50  0.0699  0.125938                1.072     0.350    0.366     1.61   
5  60  0.0992  0.032890        *       1.104     0.322    0.344     2.20   

   n_obs  
0   9463  
1   9463  
2   9463  
3   9463  
4   9463  
5   9463  
